In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt
A=df['bmi']
Anan=A[~np.isnan(A)] # Remove the NaNs
plt.boxplot(Anan);

In [ ]:
df_bmi_50 = df[df['bmi'] < 50]
df_bmi_50.head()

In [ ]:
df['bmi'].fillna(df_bmi_50['bmi'].mean(), inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df['smoking_status'].value_counts()

In [ ]:
# Unbalanced dataset
df['stroke'].value_counts()

In [ ]:
df['work_type'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.boxplot(df['age']);

In [ ]:
plt.hist(df['age']);

In [ ]:
plt.boxplot(df['avg_glucose_level']);

In [ ]:
df[df['avg_glucose_level'] == max(df['avg_glucose_level'])]

In [ ]:
df1 = df.copy()

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cols = ['gender','ever_married','work_type','Residence_type','smoking_status']
for i in cols:
    df1[i] = label_encoder.fit_transform(df1[i])

In [ ]:
df1.head()

In [ ]:
from sklearn.feature_selection import chi2

X = df1.drop(['stroke','id','age','avg_glucose_level','bmi'],axis=1)
y = df1['stroke']

In [ ]:
chi_scores = chi2(X,y)

In [ ]:
p_values = pd.Series(chi_scores[1],index = X.columns)
p_values.sort_values(ascending = False , inplace = True)

In [ ]:
p_values.plot.bar()

In [ ]:
df.head()

In [ ]:
from sklearn import preprocessing
cols = ['ever_married','work_type','Residence_type','smoking_status']
df2 = pd.get_dummies(df, columns=['ever_married','work_type','Residence_type','smoking_status'])

In [ ]:
df2.head()

In [ ]:
df2.columns

In [ ]:
X = df2[['age', 'hypertension', 'heart_disease',
       'avg_glucose_level', 'bmi', 'stroke', 'ever_married_No','work_type_Govt_job', 'work_type_Never_worked',
       'work_type_Private', 'work_type_Self-employed', 'work_type_children',
       'Residence_type_Rural', 'smoking_status_Unknown', 'smoking_status_formerly smoked',
       'smoking_status_never smoked', 'smoking_status_smokes']]
y = df2['stroke']

# Class rebalancing strategies

In [ ]:
df2.head()

In [ ]:
X = df2.drop(['stroke','id','gender'], axis=1).values

In [ ]:
y = df['stroke'].values

In [ ]:
print('Shape of Feature Matrix:', X.shape)
print('Shape of Target Vector:', y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.20, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train_sm, y_train_sm = oversample.fit_resample(X_train, y_train)

In [ ]:
sum(y_train_sm == 1)

In [ ]:
from imblearn.over_sampling import ADASYN 
ada = ADASYN(sampling_strategy='minority',random_state=42, n_neighbors=5)

In [ ]:
X_res, y_res = ada.fit_resample(X,y)

In [ ]:
from collections import Counter
print('Oversampled Target Variable Distribution:', Counter(y_res))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0, class_weight='balanced')
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
fig, ax = plt.subplots(figsize=(8, 6))
disp=plot_confusion_matrix(clf,X_test, y_test
                           , cmap=plt.cm.Blues, ax=ax)
disp.ax_.set_title("Results of the Random forest on the testing set \n with ADASYN",size=14)
plt.show()

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))